# TRATAMIENTO DE DATOS CON PYSPARK

In [ ]:
# Importar las clases necesarias de PySpark
# Si esta línea se ejecuta sin error, significa que el entorno es correcto.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# =================================================================
# 1. INICIALIZAR SPARK SESSION
# =================================================================
spark = (
    SparkSession.builder 
    .appName("FinPlus_EDA_Initial")
    .master("local[*]")
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.28") 
    .getOrCreate()
)

# Confirmación
print(f"PySpark Version: {spark.version}")
print("Spark Session iniciada con éxito. ¡Entorno listo!")